In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
path=''

users = pd.read_csv(path+'users.csv')
books = pd.read_csv(path+'books.csv')
ratings = pd.read_csv(path+'ratings.csv')

In [5]:
users.head()

,user_id,location,age
0,8,"timmins, ontario, canada",NaN
1,11400,"ottawa, ontario, canada",49.0
2,11676,"n/a, n/a, n/a",NaN
3,67544,"toronto, ontario, canada",30.0
4,85526,"victoria, british columbia, canada",36.0


In [6]:
# 칼럼 아이디 / 결측 아닌거 개수 / dtype
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68092 entries, 0 to 68091
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   user_id   68092 non-null  int64  
 1   location  68092 non-null  object 
 2   age       40259 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.6+ MB


In [7]:
users.describe().T
# 통계량 적어주는데, 전치 가능함

,count,mean,std,min,25%,50%,75%,max
user_id,68092.0,139381.329539,80523.969862,8.0,69008.75,138845.5,209388.25,278854.0
age,40259.0,36.069873,13.842571,5.0,25.00,34.0,45.00,99.0


In [8]:
users['user_id'].nunique()
# 고유한 애들 개수

68092

In [9]:
users.isna().sum()/len(users)
# 결측치 비율을 나타내는 코드
# 결측여부 데이터프레임 -> 결측치 개수 세기 -> 총 데이터 수로 나누기

user_id     0.000000
location    0.000000
age         0.408756
dtype: float64

In [11]:
users['location'] = users['location'].str.replace(r'[^0-9a-zA-Z:,]', '') # 특수문자 제거

users['location_state'] = users['location'].apply(lambda x: x.split(',')[0].strip())
users['location_state'] = users['location'].apply(lambda x: x.split(',')[1].strip())
users['location_country'] = users['location'].apply(lambda x: x.split(',')[2].strip())

users = users.replace('na', np.nan) #특수문자 제거로 n/a가 na로 바뀌게 되었습니다. 따라서 이를 컴퓨터가 인식할 수 있는 결측값으로 변환합니다.
users = users.replace('', np.nan) # 일부 경우 , , ,으로 입력된 경우가 있었으므로 이런 경우에도 결측값으로 변환합니다.

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_44048\3844664685.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  users['location'] = users['location'].str.replace(r'[^0-9a-zA-Z:,]', '') # 특수문자 제거


In [12]:
# state가 있는데 country가 결측치값인 경우는 state를 보고 채울 수 있음.

# 수정해야할 location들
# country는 결측치이면서 city가 결측치가 아닌 경우의 numpy.ndarray를 반환
modify_location = users[(users['location_country'].isna())&(users['location_state'].notnull())]['location_state'].values

location_list = []
for location in modify_location:
    try:
        right_location = users[(users['location'].str.contains(location))&(users['location_country'].notnull())]['location'].value_counts().index[0]
        location_list.append(right_location)
    except:
        pass

In [ ]:
# 도시 이름, 주, 국가
for location in location_list:
    users.loc[users[users['location_state']==location.split(',')[0]].index,'location_country'] = location.split(',')[2]